In [0]:
%run ./00_Setup

In [0]:
%sql
-- Set the active schema to strata_lab
USE SCHEMA strata_lab;

In [0]:
%sql
-- Create a text widget for the book store data path
CREATE WIDGET TEXT book_store_path DEFAULT '/Volumes/workspace/strata_lab/entrenamiento/book_store';

In [0]:
%sql
-- Create or replace the orders table from Parquet data
CREATE OR REPLACE TABLE orders AS
SELECT * FROM parquet.`${book_store_path}/orders`

In [0]:
%sql
-- Preview first 5 rows of the orders table
SELECT * FROM orders
LIMIT 5

In [0]:
%sql
-- Select orders and filter books with quantity >= 2 using FILTER function
SELECT
  order_id,
  books,
  FILTER(books, i -> i.quantity >= 2) AS multiple_copies
FROM orders

In [0]:
%sql
-- Show orders where at least one book has quantity >= 2
SELECT 
  order_id,
  multiple_copies
FROM(
  SELECT
    order_id,
    FILTER(books, i -> i.quantity >= 2) AS multiple_copies
  FROM orders
)
WHERE size(multiple_copies) > 0

In [0]:
%sql
-- Apply a 20% discount to each book's subtotal using TRANSFORM function
SELECT
  order_id,
  books,
  TRANSFORM (
    books,
    b -> CAST(b.subtotal * 0.8 AS INT)
  ) AS subtotal_after_discount
FROM orders;

In [0]:
%sql
-- Create a UDF to extract domain from email
CREATE OR REPLACE FUNCTION get_url(email STRING)
RETURNS string
RETURN CONCAT('https://www.', split(email, "@")[1])

In [0]:
%sql
-- Create or replace the customers table from JSON data
CREATE OR REPLACE TABLE customers AS
SELECT * FROM json.`${book_store_path}/customers`

In [0]:
%sql
-- Show email and extracted domain for first 5 customers
SELECT
  email,
  get_url(email) AS domain
FROM customers
LIMIT 5

In [0]:
%sql
-- Describe the get_url function
DESCRIBE FUNCTION get_url

In [0]:
%sql
-- Show extended description of get_url function
DESCRIBE FUNCTION EXTENDED get_url

In [0]:
%sql
-- Create a UDF to categorize email domain type
CREATE OR REPLACE FUNCTION site_type(email STRING)
RETURNS STRING
RETURN
  CASE
    WHEN email LIKE '%com%' THEN 'Commercial Business'
    WHEN email LIKE '%org%' THEN 'Non-profit Organization'
    WHEN email LIKE '%edu%' THEN 'Academic Institution'
    ELSE CONCAT('Unknown Site Type: ', split(email, "@")[1])
  END;

In [0]:
%sql
-- Show email and domain category for first 50 customers
SELECT
  email,
  site_type(email) AS domain_category
FROM customers
LIMIT 50

In [0]:
%sql
-- (Empty cell, no code)